![](https://www.science-accueil.org/wp-content/uploads/2021/11/assurance-maladie-logo-png.png)

The files studied here come from the French social security database.
It contains data on medications dispensed (number of boxes) and reimbursed by social security in town only (not hospital), whether they were prescribed by hospital or city prescribers.
City prescribers are independent practitioners working in private practice or clinics. While hospital prescribers work as employees.
You can find files at the folowing link : https://www.assurance-maladie.ameli.fr/etudes-et-donnees/medicaments-classe-atc-medicam

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import duckdb as db
import re

In [2]:
#Import datas
med_2021_1 = pd.read_excel('../raw_data/2021_head.xlsx', sheet_name='2021_atc2_100_et_non_a_100', skiprows=5)
med_2021_2 = pd.read_excel('../raw_data/2021_tail.xlsx', sheet_name='2021_atc2_100_et_non_a_100', skiprows=5)
med_2022_1 = pd.read_excel('../raw_data/2022_head.xlsx', sheet_name='2022_atc2_100_et_non_a_100', skiprows=5)
med_2022_2 = pd.read_excel('../raw_data/2022_tail.xlsx', sheet_name='2022_atc2_100_et_non_a_100', skiprows=5)
med_2023_1 = pd.read_excel('../raw_data/2023_head.xlsx', sheet_name='2023_atc2_100_et_non_a_100', skiprows=5)
med_2023_2 = pd.read_excel('../raw_data/2023_tail.xlsx', sheet_name='2023_atc2_100_et_non_a_100', skiprows=5)
med_2024_1 = pd.read_excel('../raw_data/2024_head.xlsx', sheet_name='2024_atc2_100_et_non_a_100', skiprows=5)
med_2024_2 = pd.read_excel('../raw_data/2024_tail.xlsx', sheet_name='2024_atc2_100_et_non_a_100', skiprows=5)

In [64]:
#Merge year
year_2021 = pd.merge(med_2021_1,med_2021_2, left_index=True, right_index=True)
year_2022 = pd.merge(med_2022_1,med_2022_2, left_index=True, right_index=True)
year_2023 = pd.merge(med_2023_1,med_2023_2, left_index=True, right_index=True)
year_2024 = pd.merge(med_2024_1,med_2024_2, left_index=True, right_index=True)

In [4]:
### à optimiser

#pd.reset_option('display.float_format')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
#pd.options.display.float_format = '{:20,.2f}'.format

In [65]:
#Function to round to 2 digits
def round_number(dataframe, decimales=2):
    for col in df.select_dtypes(include='number').columns:
        df[col] = df[col].round(decimales)

#Function to replace columns names
def replace_column_name(dataframe):
    dataframe.columns = [col_name.replace(' ', '_') for col_name in df.columns]

#Drop unnecessary columns
def drop_columns(dataframe):
    col_to_drop = ['Code_ATC2_y', 'Libellé_ATC2_y', 'Taux_de_remboursement_y']

    dataframe.drop(columns=col_to_drop, inplace = True)

#Rename columns
def rename_column(dataframe):
    col_to_rename = {'Code_ATC2_x' : 'Code_ATC2',
                     'Libellé_ATC2_x' : 'Libelle_ATC2',
                     'Taux_de_remboursement_x' : 'Taux_de_remboursement'
                     }

    dataframe.rename(columns=col_to_rename, inplace = True)

#Months
def extract_month(dataframe) :
    for df in dataframe:
        df.insert(3, 'month', df.columns[3][-2:])

#Years
def extract_year(dataframe) :
    for df in dataframe:
        df.insert(3, 'year', df.columns[4][-7:-3])

#Remove end of columns titles
def remove_end_columns(dataframe) :
    for df in dataframe :
        for col in df.columns[5:]:
            new_col_name = col[:-8].rstrip('_')
            df.rename(columns = {col: new_col_name}, inplace =True)

#Adding date columns
def ajouter_colonne_mois(dataframe):
    mois = []
    
    # Regex pour capter les 2 derniers chiffres (01 à 12) à la fin des noms de colonnes
    pattern = re.compile(r"(0[1-9]|1[0-2])$")

    # On parcourt uniquement les colonnes à partir de la 4e

    for dfind in dataframe :
        colonnes_cibles = dfind.columns[3:]

    # Transformation du DataFrame wide vers long format
    df_long = dfind.melt(id_vars=dfind.columns[:3], value_vars=colonnes_cibles,
                      var_name='nom_colonne', value_name='valeur')

    # On ajoute la colonne mois en extrayant depuis les noms de colonnes
    df_long['mois'] = df_long['nom_colonne'].str.extract(r"(0[1-9]|1[0-2])$")

    return df_long

           

In [66]:
dataframe = [year_2021, year_2022, year_2023, year_2024]
for df in dataframe:
    round_number(df, decimales=2)
    replace_column_name(df)
    drop_columns(df)
    rename_column(df)
#extract_month(dataframe)
#extract_year(dataframe)
#remove_end_columns(dataframe)
#adding_date(dataframe)

dataframe = ajouter_colonne_mois(dataframe)

In [68]:
dataframe.head()

,Code_ATC2,Libelle_ATC2,Taux_de_remboursement,nom_colonne,valeur,mois
0,A01,PREPARATIONS STOMATOLOGIQUES,100%,Base_de_remboursement_2024-01,297307.11,01
1,A01,PREPARATIONS STOMATOLOGIQUES,non à 100%,Base_de_remboursement_2024-01,3280675.41,01
2,A02,MEDICAMENTS POUR LES TROUBLES DE L'ACIDITE,100%,Base_de_remboursement_2024-01,10959327.54,01
3,A02,MEDICAMENTS POUR LES TROUBLES DE L'ACIDITE,non à 100%,Base_de_remboursement_2024-01,21115975.88,01
4,A03,MEDICAMENTS POUR LES TROUBLES FONCTIONNELS GAS...,100%,Base_de_remboursement_2024-01,1702279.72,01


In [14]:
dataframes = [year_2021, year_2022, year_2023, year_2024]
all_years = pd.concat(dataframes, axis=0)

In [46]:
def ajouter_colonne_mois(df):
    mois = []
    
    # Regex pour capter les 2 derniers chiffres (01 à 12) à la fin des noms de colonnes
    pattern = re.compile(r"(0[1-9]|1[0-2])$")

    # On parcourt uniquement les colonnes à partir de la 4e
    colonnes_cibles = df.columns[3:]

    for col in colonnes_cibles:
        match = pattern.search(col)
        if match:
            mois.append(match.group())

    # Transformation du DataFrame wide vers long format
    df_long = df.melt(id_vars=df.columns[:3], value_vars=colonnes_cibles,
                      var_name='nom_colonne', value_name='valeur')

    # On ajoute la colonne mois en extrayant depuis les noms de colonnes
    df_long['mois'] = df_long['nom_colonne'].str.extract(r"(0[1-9]|1[0-2])$")

    return df_long

ajouter_colonne_mois(year_2021)


,Code_ATC2,Libelle_ATC2,Taux_de_remboursement,nom_colonne,valeur,mois
0,A01,PREPARATIONS STOMATOLOGIQUES,100%,Base_de_remboursement_2021-01,315120.00,01
1,A01,PREPARATIONS STOMATOLOGIQUES,non à 100%,Base_de_remboursement_2021-01,3401841.00,01
2,A02,MEDICAMENTS POUR LES TROUBLES DE L'ACIDITE,100%,Base_de_remboursement_2021-01,11304609.00,01
3,A02,MEDICAMENTS POUR LES TROUBLES DE L'ACIDITE,non à 100%,Base_de_remboursement_2021-01,22669192.00,01
4,A03,MEDICAMENTS POUR LES TROUBLES FONCTIONNELS GAS...,100%,Base_de_remboursement_2021-01,1572593.00,01
...,...,...,...,...,...,...
6187,V08,PRODUITS DE CONTRASTE,100%,Montant_remboursé_\n2021-12,5845738.00,12
6188,V08,PRODUITS DE CONTRASTE,non à 100%,Montant_remboursé_\n2021-12,12097887.00,12
6189,W,Homéopathie,100%,Montant_remboursé_\n2021-12,1175.00,12
6190,W,Homéopathie,non à 100%,Montant_remboursé_\n2021-12,31006.00,12


In [50]:
from datetime import datetime

# Assuming df is your DataFrame and 'Index' is the column containing all titles
# First, convert the titles to datetime format
#year_2021['new_index'] = pd.to_datetime(year_2021['Index'], format='%Y-%m-%d')

# Then extract year and month from each title
years = [datetime.strptime(title, '%Y-%m').year for title in year_2021.columns[3][-7:-3]]
months = [datetime.strptime(title, '%Y-%m').strftime('%B') for title in year_2021.columns[3][-2:]]

ValueError: time data '2' does not match format '%Y-%m'

In [32]:
all_years['date'].unique()

<DatetimeArray>
['2021-01-01 00:00:00', '2022-01-01 00:00:00', '2023-01-01 00:00:00',
 '2024-01-01 00:00:00']
Length: 4, dtype: datetime64[ns]

In [ ]:
year_2021['day'] = 1
year_2021.insert(5, 'day', year_2021.pop('day'))
year_2021['year'] = year_2021['year'].astype(int)
year_2021['month'] = year_2021['month'].astype(int)
year_2021['day'] = year_2021['day'].astype(int)

year_2021['date'] = pd.to_datetime(year_2021[['year', 'month', 'day']])


In [17]:
year_2021.insert(3, 'date', year_2021.pop('date'))